In [4]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
import pandas as pd
import numpy as np

In [38]:
posts = pd.read_csv('Final_Posts.csv')

In [21]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

## Removing HTML tags

def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
## Removing Hashtags and @tags 

def strip_all_tags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)
## Removing special characters 

def strip_special_chars(text):
    return re.sub('\W+',' ', text)
## Removing extra white spaces

def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()
## Removing extra new lines

def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)
## Tokenize

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens
## Remove stopwords

def stopword_removal(tokenized_text):
    return [word.lower() for word in tokenized_text if word.lower() not in stopwords.words('english')]
## Remove custom stopwords

def custom_stopwords_removal(tokenized_text, filepath):
    stopwords_custom = pd.read_csv(filepath)
    stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())
    return [word.lower() for word in tokenized_text if word.lower() not in stopwords_custom.tolist()]
# Part of Speech Tagging

def get_wordnet_pos(treebank_tag):

    #if treebank_tag.startswith('J'):
     #   return wordnet.ADJ
    if treebank_tag.startswith('V'): # Elif is short for else-if
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    #elif treebank_tag.startswith('R'):
     #   return wordnet.ADV
    else:
        return "None"
# Returns part of speech of a word

def part_of_speech(temp_string):   
    tagged = nltk.pos_tag(temp_string)
    temp_pos = get_wordnet_pos(tagged[0][1])
    
    return temp_pos

    del tagged,temp_pos  
# Lemmatizes the word

def lemmatize(temp_word, temp_pos):
    wordnet_lemmatizer = WordNetLemmatizer()
    if temp_pos == "None":
        temp_lemma = wordnet_lemmatizer.lemmatize(temp_word)
    else:
        temp_lemma = wordnet_lemmatizer.lemmatize(temp_word,temp_pos)
    return temp_lemma
    del temp_pos, temp_word, temp_lemma
## Lemmatization

def lemmatize_tokens(tokenized_text, withpos):
    lemmatized_list = []
    if withpos:
        for word in tokenized_text:
            word = word.lower()
            word_pos = part_of_speech(word)
            print(word_pos)
            lemmatized_list.append(lemmatize(word, word_pos))
    else:
        for word in tokenized_text:
            word = word.lower()
            lemmatized_list.append(lemmatize(word,"None"))
    return lemmatized_list
    

In [22]:
## Preprocessing
def preprocessing
customstopwords = 'C:\\Users\\Varun.Behl\\Documents\\stopwords1.csv'
filtered_posts=posts['posts_p'].map(lambda x:cleanhtml(x))
filtered_posts=filtered_posts.map(lambda x:strip_URL(x))
filtered_posts=filtered_posts.map(lambda x:strip_all_tags(x))
filtered_posts=filtered_posts.map(lambda x:strip_special_chars(x))
filtered_posts=filtered_posts.map(lambda x:strip_extra_white_spaces(x))
filtered_posts=filtered_posts.map(lambda x:strip_extra_new_lines(x))
tokens=filtered_posts.map(lambda x:tokenize(x))
tokens_swremoval=tokens.map(lambda x:custom_stopwords_removal(x,customstopwords))
lemmas=tokens_swremoval.map(lambda x:lemmatize_tokens(x,False))
A=[]
for item in lemmas:
    a= " ".join(str(x) for x in item)
    A.append(a)
return A
    

In [45]:
import gensim
model = gensim.models.Word2Vec(lemmas, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

ImportError: No module named gensim

In [31]:
type(lemmas)

pandas.core.series.Series

In [10]:
import string
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.cross_validation import KFold

In [11]:
lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and @tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)

def inverse_transform(l):
        sent = ''
        for word in l:
            sent += word+" "
        return [sent]

def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())
        
    # Break the document into sentences
    for sent in sent_tokenize(document):
        # Break the sentence into part of speech tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower() 

            # If stopword, ignore token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If punctuation, ignore token and continue
            if all(char in string.punctuation for char in token):
                continue
            
            yield token

            # Lemmatize the token and yield
#             lemma = lemmatize(token, tag)
#             yield lemma

def lemmatize(token, tag):
    tag = {
        'N': wn.NOUN,
        'V': wn.VERB,
       # 'R': wn.ADV,
        #'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    return lemmatizer.lemmatize(token, tag)
    
def preprocessing(X):
    tokens_dict = {}
    count = 0
    for doc in X:
        tokens = list(tokenize(doc))
        tokens_dict[count] = tokens
#         tokens_dict[count] = inverse_transform(tokens)
        count += 1
    return tokens_dict


In [20]:

preprocessing(posts['posts_p'])


TypeError: bad operand type for unary -: 'dict'

In [16]:
tokens_dict

NameError: name 'tokens_dict' is not defined

In [27]:
A=[]
for item in lemmas:
    a= " ".join(str(x) for x in item)
    A.append(a)
    
for item in A:
    string_words= ' '.join(map(str,A))

    
    
    

In [ ]:
#type(y_test)
#preprocessed_train = preprocessing(X_train)
#string_words= ' '.join(map(str,preprocessed_train))
    
#A =[]
#for i in preprocessed_train:
   # a = " ".join(str(x) for x in i)
# A.append(a) 

In [43]:
type(A)

list

In [ ]:
for item in A:
    string_words= ' '.join(map(str,A))
    print string_words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(A)
sklearn_representation.toarray()
sklearn_tfidf.get_feature_names()

In [50]:
A=[]
L = [['1','2','3'],['4','5','6'],['7','8','9']]
for item in L:
    a= " ".join(str(x) for x in item)
    print a
    A.append(a)
    type(A)

1 2 3
4 5 6
7 8 9


In [51]:

for item in A:
    string_words= ' '.join(map(str,A))
print string_words

1 2 3 4 5 6 7 8 9


In [4]:
for item in A:
    print item.strip('[]')

1 2 3
4 5 6
7 8 9


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=True, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(A)
sklearn_representation.toarray()



#saving the model(sklearn.feature_extraction)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

NameError: name 'pickle' is not defined

In [73]:
lemmas_list = lemmas.tolist()

In [6]:
from gensim.models import Word2Vec
 
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = Word2Vec.Text8Corpus('A')
 
model = word2vec.Word2Vec(sentences, size=200)

AttributeError: type object 'Word2Vec' has no attribute 'Text8Corpus'

SyntaxError: invalid syntax (<ipython-input-32-f59e656ba699>, line 1)

In [33]:
! pip install sklearn

  Running setup.py bdist_wheel for sklearn: started
  Running setup.py bdist_wheel for sklearn: finished with status 'done'
  Stored in directory: C:\Users\Arpita.Sharma\AppData\Local\pip\Cache\wheels\d7\db\a3\1b8041ab0be63b5c96c503df8e757cf205c2848cf9ef55f85e
Successfully built sklearn


In [39]:
from sklearn.feature_selection import VarianceThreshold
#X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(L)


array([['1', '2', '3'],
       ['4', '5', '6'],
       ['7', '8', '9']], 
      dtype='|S1')

In [20]:
sklearn_tfidf.get_feature_names()

[u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9']

In [60]:
lemmas = posts["posts_tokens_lemma"].tolist()

In [61]:
print(len(lemmas[0]))
print(len(lis[0].split(" ")))

205
21


In [53]:
len(np.where(tfidf.toarray()[0,:]!=0)[0])

17

In [ ]:

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(A)
indices = np.argsort(vectorizer.idf_)[::1]
features = vectorizer.get_feature_names()
top_n = 50
top_features = [features[i] for i in indices[:top_n]]
print top_features



In [29]:
len(string_words.split())

187417

In [31]:
from collections import defaultdict
y = len(string_words.split())
z = int(round(y*0.2))
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(A)
features_by_gram = defaultdict(list)

for f, w in zip(vectorizer.get_feature_names(), vectorizer.idf_):
    features_by_gram[len(f.split(' '))].append((f, w))
top_n = z
for gram, features in features_by_gram.iteritems():
    top_features = sorted(features, key=lambda x: x[1], reverse=False)[:top_n]
    top_features = [f[0] for f in top_features]




In [4]:
import os
os.getcwd()

'C:\\Users\\varun.behl\\Documents'

In [ ]:
z = y *0.5
int(round(z,0))

In [ ]:
temporal_list = pd.read_csv("temporal_words.csv")
temporal_list = temporal_list.ix[:,0].map(lambda x:x.lower())
def TemporalEntitiesCount(X):
    count_list = []
    for document in X:
        count = 0
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token in wordpunct_tokenize(sent):
                if token in temporal_list.tolist():
                    count += 1 
        count_list.append(count)
#     return np.asarray(count_list)
    return np.reshape(np.asarray(count_list),(len(X),1))


In [41]:
feature = top_features
type(feature)

list

In [39]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse

X = posts["posts_p"]
y = posts["TAG"]

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=111)


In [42]:
classifier = LinearSVC()
model = classifier.fit(feature, y_train)

preprocessed_test = preprocessing(X_test)
tfs_test = tfidf.transform(preprocessed_test.values())

TempCount_test = TemporalEntitiesCount(X_test)
print("SVM classifier")

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
feature_test = tfs_test

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

ValueError: could not convert string to float: side effect

In [ ]:
tfidf_mean = np.asarray(X_1.mean(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'tfidf': tfidf_mean})
counts_df.sort_values(by='tfidf', ascending=False).head(100)    # Mean - Final